In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.utils import to_categorical

In [ ]:
raw_data=pd.read_excel('엑셀파일') # 엑셀파일 불러와서 정리안된 아래 기술지표함수 참고하여 추가  (DX, ADX, MFI, williams r, OBV 등 몇개 아직 미완료 )

In [ ]:
# Data preprocessing 기술지표관련 
# volumne variance 큰것 분류해서 따로 분석필요

def ADX(data, period):  
    df = data[['Close','High','Low','TR']].copy()
    alpha = 1/period
    df['ATR'] = df['TR'].ewm(alpha=alpha, adjust=False).mean()

    # +-DX
    df['H-pH'] = df['High'] - df['High'].shift(1)
    df['pL-L'] = df['Low'].shift(1) - df['Low']
    df['+DX'] = np.where(
        (df['H-pH'] > df['pL-L']) & (df['H-pH']>0),
        df['H-pH'], 0.0    )
    df['-DX'] = np.where(
        (df['H-pH'] < df['pL-L']) & (df['pL-L']>0),
        df['pL-L'], 0.0    )
    del df['H-pH'], df['pL-L']

    # +- DMI
    df['S+DM'] = df['+DX'].ewm(alpha=alpha, adjust=False).mean()
    df['S-DM'] = df['-DX'].ewm(alpha=alpha, adjust=False).mean()
    df['+DMI'] = (df['S+DM']/df['ATR'])*100
    df['-DMI'] = (df['S-DM']/df['ATR'])*100
    del df['S+DM'], df['S-DM']

    # ADX
    df['DX'] = (np.abs(df['+DMI'] - df['-DMI'])/(df['+DMI'] + df['-DMI']))*100
    df['ADX'] = df['DX'].ewm(alpha=alpha, adjust=False).mean()
    del df['ATR'], df['-DX'], df['+DX'], df['+DMI'], df['-DMI']
    return df['DX'],df['ADX']

def rsi_n(data,n):
  delta=np.array(data.diff(1))
  delta[np.isnan(delta)]=0.000000001
  dUp, dDown = delta.copy(), delta.copy()
  dUp[dUp < 0] = 0
  dDown[dDown > 0] = 0
  RolUp = pd.DataFrame(dUp).rolling(n).mean()
  RolDown = pd.DataFrame(dDown).abs().rolling(n).mean()
  RS =RolUp / RolDown
  rsi= 100.0 - (100.0 / (1.0 + np.array(RS)))
  return rsi

def forceindex(data_c,data_v,ndays):
    ForceIndex=pd.Series(data_c.diff(1)* data_v)    
    return ForceIndex.ewm(ndays).mean()


def CCI(data_tp, ndays): 
    CCI = pd.Series((data_tp - data_tp.rolling(ndays).mean()) / (0.015 * data_tp.rolling(ndays).std()),name = 'CCI') 
    return CCI 

def money_flow_index(df, periods=14):
    data=df.copy()
    data['money_flow'] = np.array(data['TP']) * np.array(data['Volume'])
    # data['money_flow'] = data['TP'] * data['Volume']
    delta=data['money_flow'].diff(1)
    delta[np.isnan(delta)]=0.000000001
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0
    RolUp = pd.DataFrame(dUp).rolling(periods).mean()
    RolDown = pd.DataFrame(dDown).abs().rolling(periods).mean()
    MFR =RolUp / RolDown
    MFI= 100.0 - (100.0 / (1.0 + np.array(MFR)))
    return MFR, MFI

def negative_volume_index(data, periods=255, close_col='<CLOSE>', vol_col='<VOL>'):
    data['nvi'] = 0.
    
    for index,row in data.iterrows():
        if index > 0:
            prev_nvi = data.at[index-1, 'nvi']
            prev_close = data.at[index-1, close_col]
            if row[vol_col] < data.at[index-1, vol_col]:
                nvi = prev_nvi + (row[close_col] - prev_close / prev_close * prev_nvi)
            else: 
                nvi = prev_nvi
        else:
            nvi = 1000
        data.set_value(index, 'nvi', nvi)
    data['nvi_ema'] = data['nvi'].ewm(ignore_na=False, min_periods=0, com=periods, adjust=True).mean()
    
    return data


def williams_r(data, periods=14, high_col='<HIGH>', low_col='<LOW>', close_col='<CLOSE>'):
    data['williams_r'] = 0.
    
    for index,row in data.iterrows():
        if index > periods:
            data.set_value(index, 'williams_r', ((max(data[high_col][index-periods:index]) - row[close_col]) / 
                                                 (max(data[high_col][index-periods:index]) - min(data[low_col][index-periods:index]))))
        
    return data


def on_balance_volume(data, trend_periods=21, close_col='<CLOSE>', vol_col='<VOL>'):
    for index, row in data.iterrows():
        if index > 0:
            last_obv = data.at[index - 1, 'obv']
            if row[close_col] > data.at[index - 1, close_col]:
                current_obv = last_obv + row[vol_col]
            elif row[close_col] < data.at[index - 1, close_col]:
                current_obv = last_obv - row[vol_col]
            else:
                current_obv = last_obv
        else:
            last_obv = 0
            current_obv = row[vol_col]

        data.set_value(index, 'obv', current_obv)

    data['obv_ema' + str(trend_periods)] = data['obv'].ewm(ignore_na=False, min_periods=0, com=trend_periods, adjust=True).mean()
    
    return data


class EaseOfMovementIndicator(IndicatorMixin):

    def __init__(
        self,
        high: pd.Series,
        low: pd.Series,
        volume: pd.Series,
        window: int = 14,
        fillna: bool = False,
    ):
        self._high = high
        self._low = low
        self._volume = volume
        self._window = window
        self._fillna = fillna
        self._run()

    def _run(self):
        self._emv = (
            (self._high.diff(1) + self._low.diff(1))
            * (self._high - self._low)
            / (2 * self._volume)
        )
        self._emv *= 100000000

    def ease_of_movement(self) -> pd.Series:
        """Ease of movement (EoM, EMV)
        Returns:
            pandas.Series: New feature generated.
        """
        emv = self._check_fillna(self._emv, value=0)
        return pd.Series(emv, name=f"eom_{self._window}")

    def sma_ease_of_movement(self) -> pd.Series:
        """Signal Ease of movement (EoM, EMV)
        Returns:
            pandas.Series: New feature generated.
        """
        min_periods = 0 if self._fillna else self._window
        emv = self._emv.rolling(self._window, min_periods=min_periods).mean()
        emv = self._check_fillna(emv, value=0)
        return pd.Series(emv, name=f"sma_eom_{self._window}")


In [ ]:
# 2000-01-03 / 2019-07-01
# 2007-01-03 / 2011-12-30
## target논문 데이터 기간


train_x=snp_x.loc['2007-01-03':'2011-01-02']
test_x=snp_x.loc['2011-01-03':'2011-12-30']

train_y=snp_y.loc['2007-01-03':'2011-01-02'] ## 논문에서 제시한 train기간
test_y=snp_y.loc['2011-01-03':'2011-12-30']  ## 논문에서 제시한 test기간
  



In [ ]:
#scaler=StandardScaler()
scaler=MinMaxScaler()  # 0~1 nomarlization (input전에 전체데이터 전처리)
scaler.fit(np.array(train_x))

train_x_n=scaler.transform(train_x)
test_y_n=scaler.transform(test_x)


train_y_return=train_y['ln_return']
train_y_class=to_categorical(train_y['up_down'])

test_y_return=test_y['ln_return']
test_y_class=to_categorical(test_y['up_down'])


##### total
data_y=pd.concat([train_y,test_y])
data_x=pd.concat([train_x,test_x])

data_x_n=scaler.transform(data_x)

data_y_return=np.array(data_y['ln_return'])
data_y_class=to_categorical(data_y['up_down'])



train_section=len(test_x)
print(len(data_x_n),len(train_x),len(test_x))
print(test_y_return.shape,train_y.shape)

In [ ]:
import os
import tensorflow as tf
import random, sys, random, pickle
from keras.models import Sequential, Model
from keras import optimizers
from keras.layers import Input, LSTM, Dense, Dropout
from keras.regularizers import l2, l1
from keras.layers import Reshape, Activation, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint,LearningRateScheduler, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import keras.backend as K
from pandas_datareader import data as pdr
from tensorflow.python.client import device_lib



def create_timestep(dataset,n):
    dataX= []
    for i in range(len(dataset)-n+1):
        a = dataset[i:(i+n)]
        dataX.append(a)
    return np.array(dataX)
def count(data):  
    c0=0
    c1=0
    for i in range(len(data)):
        if np.argmax(data[i])==0:
            c0=c0+1
        elif np.argmax(data[i])==1:
            c1=c1+1
    return c0,c1
def label(data):
    k=[]
    for i in range(len(data)):
        if np.argmax(data[i])==0:
            k.append(0)
        elif np.argmax(data[i])==1:
            k.append(1)
    return np.array(k)


drop_rate=0.5
initial_rate=0.001
step_epoch=50
lrList = []
def step_decay(epoch):
    lrate = initial_rate
    if epoch >= step_epoch:
        lrate = initial_rate*drop_rate
    elif epoch >=step_epoch*2:
        lrate = lrate*drop_rate
    elif epoch >=step_epoch*3:
        lrate = lrate*drop_rate
    lrList.append(lrate)
    return lrate



def lstm_forest_model(var,index,data,seq):
  for i in range(var):
    if i==0:
      lf_datax=create_timestep(data[:,index[0]],seq)      
      lf_datax=lf_datax.reshape((-1,seq,1))      
    else:
      lf_datax_i=create_timestep(data[:,index[i]],seq).reshape((-1,seq,1))      
      lf_datax=np.concatenate((lf_datax,lf_datax_i),axis=2)           
  return lf_datax
        



def test_LFM(data_x,y_return,y_cross,train_section):
  
  var=13
  seq=50
  epoch=150

  lis=range(data_x.shape[1])  
  index=random.sample(lis,var)    
  print('index',index)
  lf_datax=lstm_forest_model(var,index,data_x,seq)  
  lr = LearningRateScheduler(step_decay)
  callbacks_list = [lr]
  section=train_section
  
  y_cross, y_return =y_cross[seq-1:], y_return[seq-1:]
  train_x2, test_x = lf_datax[:-section,:,:], lf_datax[-section:,:,:]
  train_y_cross2, test_y_cross=y_cross[:-section],y_cross[-section:]
  train_y_return2, test_y_return=y_return[:-section],y_return[-section:]  
  
  #train sample random choice
  index_random2=random.sample(range(train_x2.shape[0]),int(train_x2.shape[0]*7/8))  
  index_random=np.sort(index_random2)

  train_x=train_x2[index_random,:,:]    
  train_y_cross=train_y_cross2[index_random,:]    
  train_y_return=train_y_return2[index_random]
  # train_y_return=train_y_return2[index_random,:]
  
  # modeling
  K.clear_session() 
  main_input = Input(shape=(seq,var),name='main_input')
  m=LSTM(15,return_sequences=False)(main_input)
  
  m=Dense(40,activation='relu',kernel_initializer='he_normal')(m)
  m=Dense(40,activation='relu',kernel_initializer='he_normal')(m)
  m=Dense(40,activation='relu',kernel_initializer='he_normal')(m)
  m=Dropout(0.3)(m)

  main_cross=Dense(30,activation='relu',kernel_initializer='he_normal')(m)
  main_cross=Dense(30,activation='relu',kernel_initializer='he_normal')(main_cross)
  main_cross=Dropout(0.3)(m)
  main_cross=Dense(30,activation='relu',kernel_initializer='he_normal')(main_cross)
  main_cross=Dense(2,activation='softmax',name='main_cross')(main_cross)

  main_return=Dense(30,activation='relu',kernel_initializer='he_normal')(m)
  main_return=Dense(30,activation='relu',kernel_initializer='he_normal')(main_return)
  main_return=Dropout(0.3)(main_return)
  main_return=Dense(30,kernel_initializer='he_normal')(main_return)
  main_return=Dense(1,activation='linear',name='main_return')(main_return)
  # model.summary()
  model=Model(inputs=main_input,outputs=[main_cross,main_return])
  model.compile(optimizer='adam',loss={'main_cross':'categorical_crossentropy','main_return':'mean_squared_error'},
                  metrics=['accuracy'],loss_weights={'main_cross':8,'main_return':1})
  print("model : ",model.summary())
  history=model.fit({'main_input':train_x},{'main_cross':train_y_cross,'main_return':train_y_return},shuffle=False,callbacks=callbacks_list,
                    validation_split=1/8,epochs=epoch,batch_size=256,verbose=0)
  
  # model evaluation
  lfm_predict=model.predict(test_x)

  val_section=int(len(train_x)*1/8)
  val_result=model.evaluate(train_x[-val_section:],[train_y_cross[-val_section:],train_y_return[-val_section:]])
  print("test evaluate",model.evaluate(test_x,[test_y_cross,test_y_return]))
  print(len(lfm_predict[0]),len(lfm_predict[1]))
  model.save('save dir + stockanme+ num_var+ sequence+ -th lstm')  ## ex :' /user/hyunjun/asc/lstm_forest/lfm/snp_v13_seq50_lstm{}' .format(p)    p= 아래 몇번째 LSTM 인지 
  
  return lfm_predict,val_result




lfm_var13_se50=[]

import pickle
for p in range(103):
  print(p ,'-th complete')
  lfm_var13_se50.append(test_LFM(data_x_n,data_y_return,data_y_class,train_section))            

# 최종 predidction : 100개의 lfm model로부터 prediction하고 예측값 앙상블한결과
